# Smoothing the curve 📈

## From dictionaries to tables

Let's recap the concept of dictionary and how can we generate one:

In [1]:
`a`b!1 2

a| 1
b| 2


When a dictionary contains lists of equal length, and the keys are symbols, we create a *column dictionary*.

In [3]:
d: `a`b!(til 4;til 4)

We can convert a column dictionary into a table by applying `flip`. This turns the dictionary into a table where keys become column names and values become the rows.

In [5]:
flip d

a b
---
0 0
1 1
2 2
3 3


<div class="alert alert-warning">
One of the keys of why kdb is so efficient is that kdb database is column oriented rather than row-oriented, as in traditional relational databases like SQL. This means data is stored by columns, not rows.
</div>

<div class="alert alert-success">
In kdb+, tables are <strong>first-class citizens</strong>, meaning they can be passed to functions, returned, and assigned to variables, just like lists or any other datatype. 
</div>

## Generating items

Going back to our problem, we need to gen a price tabl that look like this:

<img src="../pics/priceTable.JPG" alt="Italian Trulli">

What do we need to gen one example of our price table?
- Temporal column
- Asset column
- Price column



To build the temporal column we have the built-in `minute` type, that simplifies working with time values, allowing you to perform operations directly on hours.

In [7]:
9#09:00 + til 3

09:00 09:01 09:02 09:00 09:01 09:02 09:00 09:01 09:02


If you look closely you can realise that we need something more, we need to sort ascendelly the hours, so we are going to use `asc` keyword to handle that. And, then we store our list in `ts`

In [9]:
ts: asc 9#09:00 + til 3

<div class="alert alert-warning">
Look a the `s#` at the beginning of the output. It means that, that list has the <strong>sorted</strong> atribbute, meaning the data is recognized by q as being in sorted order, which allows for faster lookups and optimizations during queries
</div>

Now, to generate our list of assets, we are going to use *symbols*. Symbols are atomic data types rather than lists of characters, unlike strings. This makes them more efficient for use in tables, as symbols store references to their unique value rather than the full string.

So similarly, we can create a list of symbols for the assets and store it in `syms`

In [10]:
syms: 9#`AAPL`AMZN`GOOG

Finally, we are going to need to generate random data, lets use `roll` `?`, for example to roll a dice 10 times

In [12]:
1 + 10?6

5 6 2 1 6 3 5 6 2 3


Left number is the number of times and right number is the limit number, so let's first generate starting prices 

In [13]:
3?100f

23.06385 94.9975 43.9081


And then we generate randomly the price jumps between each instant of time. In order to sum the jumps to the starting points, we need to reshape the starting points, so both lists conform

In [14]:
(9?0.05) + 9#3?100f

57.60997 59.20962 84.81973 57.63735 59.20395 84.82763 57.59805 59.19788 84.86..


As last point, random data is generated based on a seed value, which controls the sequence of random numbers. By fixing the seed using `\S`, we ensure that the same random output is reproducible across different runs.

In [15]:
\S 7
p: (9?0.05) + 9#3?100f

Now, we have the ingredients (variables `ts`, `syms` and `p`) to create our column dictionary ...

In [17]:
d: `t`sym`price!(ts;syms;p)

... and our table will be ready just flipping the dictionary

In [18]:
t1: flip d

Finally, there's another syntax to create a table

In [19]:
([]t:ts;sym:syms;price:p)

t     sym  price   
-------------------
09:00 AAPL 96.26213
09:00 AMZN 78.44964
09:00 GOOG 75.3932 
09:01 AAPL 96.27035
09:01 AMZN 78.42006
09:01 GOOG 75.3768 
09:02 AAPL 96.27845
09:02 AMZN 78.42342
09:02 GOOG 75.37487


### SQL in the Q world

By now, we know what a table is and how to create one in q. Next, we'll explore what we can do with tables using qSQL, a set of SQL-like functions in q. These operations will feel familiar if you have SQL experience.

<div class="alert alert-info">
Using your previous SQL knowledge, how would you select all columns from a table?
<div>

In [20]:
select from t1

t     sym  price   
-------------------
09:00 AAPL 96.26213
09:00 AMZN 78.44964
09:00 GOOG 75.3932 
09:01 AAPL 96.27035
09:01 AMZN 78.42006
09:01 GOOG 75.3768 
09:02 AAPL 96.27845
09:02 AMZN 78.42342
09:02 GOOG 75.37487


Filtering rows is just as simple.

In [21]:
select from t1 where sym=`AAPL

t     sym  price   
-------------------
09:00 AAPL 96.26213
09:01 AAPL 96.27035
09:02 AAPL 96.27845


We can also create new columns or select specific ones. Here's an example of calculating the average price for `AAPL`

In [22]:
select avg price from t1

price   
--------
83.36099


In qSQL, the `update` statement only creates new columns or overwrites existing ones without modifying the original table structure. For example, let's add a "mean" column to our table.

In [23]:
update mean:avg price from t1

t     sym  price    mean    
----------------------------
09:00 AAPL 96.26213 83.36099
09:00 AMZN 78.44964 83.36099
09:00 GOOG 75.3932  83.36099
09:01 AAPL 96.27035 83.36099
09:01 AMZN 78.42006 83.36099
09:01 GOOG 75.3768  83.36099
09:02 AAPL 96.27845 83.36099
09:02 AMZN 78.42342 83.36099
09:02 GOOG 75.37487 83.36099


Grouping and aggregating is also straightforward using the `by` clause, similar to SQL's `GROUP BY`.

In [24]:
select avg price by sym from t1

sym | price   
----| --------
AAPL| 96.27031
AMZN| 78.43104
GOOG| 75.38162


Hmm, what's that vertical line we see in our grouped table?

<div class="alert alert-warning">
This is a <code>keyed table</code> , which is the way to implement <strong>primary keys</strong> in kdb+. While this gives a basic idea, it's not the formal definition—the concept is more complex than just that.
</div>

## Getting previous element

Focusing back in the problem, we aim to compute the average of the current price with the previous price for each symbol. To access the previous value in a sequence, we can use the `prev` keyword in q. Let's see how it works.

In [25]:
l: 100 120 150
(l + prev l)%2

0n 110 135


Notice that the first element is `0N` (null) since there's no preceding value. To avoid this, we can use the `fill` (`^`) keyword to fill the nulls with appropriate values

In [26]:
l^(l + prev l)%2

100 110 135f


Let's now integrate this logic with our table operations using qSQL syntax to compute the average for each symbol's prices.

In [27]:
update price:price^(price+prev price)%2 from t1

t     sym  price   
-------------------
09:00 AAPL 96.26213
09:00 AMZN 87.35589
09:00 GOOG 76.92142
09:01 AAPL 85.83178
09:01 AMZN 87.34521
09:01 GOOG 76.89843
09:02 AAPL 85.82762
09:02 AMZN 87.35093
09:02 GOOG 76.89915


Finally, we just need to encapsulate it in a function

In [28]:
f: {update price:price^(price+prev price)%2 from x}

We got it!

## InterConnection

kdb+ supplies interprocess communication (IPC), which enables communication between different q processes. To demonstrate this, we need to start the server side of a q process.

We can begin by listening for incoming requests by using `\p` followed by a port number.

In [29]:
\p 5001